In [2]:
## Find the exact bounds for \alpha_i for a variance normalized (not mean) distr
p, n, u, K = var('p, n, u, K')
errorTerm = 0.56 / (sqrt(p) * (1-p) ^ (1.5) * sqrt(n))
mean = 0#n*p / sqrt(p * (1 - p) * n)
guassian = 1 / sqrt(2 * pi) * e ^ (-0.5 * (u - mean)^2)
# eqSmaller = -errorTerm + (1/u + 1/u^3) * guassian <= K / n
eqLarger = errorTerm + (1/u) * guassian == K / n
((solve([eqLarger], u)))

[u == 25*n^(3/2)*sqrt(p)*(-p + 1)^(3/2)*e^(-1/2*u^2)/(25*sqrt(2)*sqrt(pi)*K*sqrt(n)*sqrt(p)*(-p + 1)^(3/2) - 14*sqrt(2)*sqrt(pi)*n)]

In [29]:
uSimUpperBound = sqrt(2 * ln(25*n^(3/2)*sqrt(p)*(-p + 1)^(3/2)/(25*sqrt(2)*sqrt(pi)*K*sqrt(n)*sqrt(p)*(-p + 1)^(3/2) - 14*sqrt(2)*sqrt(pi)*n)))

# Denormalize
(uSimUpperBound * sqrt(n * p * (1-p)) + n * p)(n=1000, p = 0.01, K = 1).n() 

17.1330927888582 + 4.36020786370807*I